In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from pylab import savefig
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.linear_model import LinearRegression
import pickle

In [2]:
jan2019 = pd.read_csv("./2019 data/201901-capitalbikeshare-tripdata.csv")
feb2019 = pd.read_csv("./2019 data/201902-capitalbikeshare-tripdata.csv")
mar2019 = pd.read_csv("./2019 data/201903-capitalbikeshare-tripdata.csv")
apr2019 = pd.read_csv("./2019 data/201904-capitalbikeshare-tripdata.csv")
may2019 = pd.read_csv("./2019 data/201905-capitalbikeshare-tripdata.csv")
jun2019 = pd.read_csv("./2019 data/201906-capitalbikeshare-tripdata.csv")
#jul2019 = pd.read_csv("./2019 data/201907-capitalbikeshare-tripdata.csv")
aug2019 = pd.read_csv("./2019 data/201908-capitalbikeshare-tripdata.csv")
sep2019 = pd.read_csv("./2019 data/201909-capitalbikeshare-tripdata.csv")
oct2019 = pd.read_csv("./2019 data/201910-capitalbikeshare-tripdata.csv")
nov2019 = pd.read_csv("./2019 data/201911-capitalbikeshare-tripdata.csv")
dec2019 = pd.read_csv("./2019 data/201912-capitalbikeshare-tripdata.csv")

In [3]:
y2019 = pd.concat([jan2019,feb2019,mar2019,apr2019,may2019,jun2019,aug2019,sep2019,oct2019,nov2019,dec2019], axis=0)
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member
...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member


In [4]:
y2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3041772 entries, 0 to 161877
Data columns (total 9 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Duration              int64 
 1   Start date            object
 2   End date              object
 3   Start station number  int64 
 4   Start station         object
 5   End station number    int64 
 6   End station           object
 7   Bike number           object
 8   Member type           object
dtypes: int64(3), object(6)
memory usage: 232.1+ MB


In [5]:
y2019['Start date'] = pd.to_datetime(y2019['Start date'])

y2019['date'] = y2019['Start date'].dt.date

y2019['time'] = y2019['Start date'].dt.time
y2019.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29


In [6]:
y2019['month'] = y2019['Start date'].dt.month
y2019.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,1
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,1
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,1
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,1
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,1


In [7]:
y2019.loc[y2019['month'] == 1, 'month'] = 'January'
y2019.loc[y2019['month'] == 2, 'month'] = 'February'
y2019.loc[y2019['month'] == 3, 'month'] = 'March'
y2019.loc[y2019['month'] == 4, 'month'] = 'April'
y2019.loc[y2019['month'] == 5, 'month'] = 'May'
y2019.loc[y2019['month'] == 6, 'month'] = 'June'
y2019.loc[y2019['month'] == 7, 'month'] = 'July'
y2019.loc[y2019['month'] == 8, 'month'] = 'August'
y2019.loc[y2019['month'] == 9, 'month'] = 'September'
y2019.loc[y2019['month'] == 10, 'month'] = 'October'
y2019.loc[y2019['month'] == 11, 'month'] = 'November'
y2019.loc[y2019['month'] == 12, 'month'] = 'December'

In [8]:
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,January
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,January
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,January
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,January
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,January
...,...,...,...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member,2019-12-31,23:57:36,December
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member,2019-12-31,23:57:47,December
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member,2019-12-31,23:59:37,December
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,2019-12-31,23:59:38,December


In [9]:
y2019['weekday'] = y2019['Start date'].dt.weekday
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month,weekday
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,January,1
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,January,1
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,January,1
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,January,1
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,January,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member,2019-12-31,23:57:36,December,1
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member,2019-12-31,23:57:47,December,1
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member,2019-12-31,23:59:37,December,1
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,2019-12-31,23:59:38,December,1


In [10]:
y2019.loc[y2019['weekday'] == 1, 'weekday'] = 'Monday'
y2019.loc[y2019['weekday'] == 2, 'weekday'] = 'Tuesday'
y2019.loc[y2019['weekday'] == 3, 'weekday'] = 'Wednesday'
y2019.loc[y2019['weekday'] == 4, 'weekday'] = 'Thursday'
y2019.loc[y2019['weekday'] == 5, 'weekday'] = 'Friday'
y2019.loc[y2019['weekday'] == 6, 'weekday'] = 'Saturday'
y2019.loc[y2019['weekday'] == 7, 'weekday'] = 'Sunday'

In [11]:
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month,weekday
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,January,Monday
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,January,Monday
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,January,Monday
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,January,Monday
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,January,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member,2019-12-31,23:57:36,December,Monday
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member,2019-12-31,23:57:47,December,Monday
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member,2019-12-31,23:59:37,December,Monday
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,2019-12-31,23:59:38,December,Monday


In [12]:
y2019.drop

<bound method DataFrame.drop of         Duration          Start date             End date  \
0            230 2019-01-01 00:04:48  2019-01-01 00:08:39   
1           1549 2019-01-01 00:06:37  2019-01-01 00:32:27   
2            177 2019-01-01 00:08:46  2019-01-01 00:11:44   
3            228 2019-01-01 00:08:47  2019-01-01 00:12:35   
4           1300 2019-01-01 00:12:29  2019-01-01 00:34:10   
...          ...                 ...                  ...   
161873       130 2019-12-31 23:57:36  2019-12-31 23:59:46   
161874       664 2019-12-31 23:57:47  2020-01-01 00:08:51   
161875       389 2019-12-31 23:59:37  2020-01-01 00:06:06   
161876       962 2019-12-31 23:59:38  2020-01-01 00:15:40   
161877       183 2019-12-31 23:59:55  2020-01-01 00:02:58   

        Start station number                           Start station  \
0                      31203              14th & Rhode Island Ave NW   
1                      31321           15th St & Constitution Ave NW   
2                  

In [13]:
y2019.loc[y2019['weekday'] == 'Saturday', 'workingday'] = 'weekend'
y2019.loc[y2019['weekday'] == 'Sunday', 'workingday'] = 'weekend'
y2019.loc[y2019['weekday'] == 'Monday', 'workingday'] = 'weekday'
y2019.loc[y2019['weekday'] == 'Tuesday', 'workingday'] = 'weekday'
y2019.loc[y2019['weekday'] == 'Wednesday', 'workingday'] = 'weekday'
y2019.loc[y2019['weekday'] == 'Thursday', 'workingday'] = 'weekday'
y2019.loc[y2019['weekday'] == 'Friday', 'workingday'] = 'weekday'

In [14]:
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month,weekday,workingday
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,January,Monday,weekday
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,January,Monday,weekday
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,January,Monday,weekday
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,January,Monday,weekday
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,January,Monday,weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member,2019-12-31,23:57:36,December,Monday,weekday
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member,2019-12-31,23:57:47,December,Monday,weekday
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member,2019-12-31,23:59:37,December,Monday,weekday
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,2019-12-31,23:59:38,December,Monday,weekday


In [15]:
y2019['workingday'].value_counts()

weekday    2272546
weekend     359940
Name: workingday, dtype: int64

In [16]:
holiday = pd.date_range(start='2019-01-01', end='2019-12-31')

cal = calendar()
holidays = cal.holidays(start=holiday.min(), end=holiday.max())

y2019['Holiday'] = y2019['date'].isin(holidays)
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month,weekday,workingday,Holiday
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,January,Monday,weekday,False
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,January,Monday,weekday,False
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,January,Monday,weekday,False
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,January,Monday,weekday,False
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,January,Monday,weekday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member,2019-12-31,23:57:36,December,Monday,weekday,False
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member,2019-12-31,23:57:47,December,Monday,weekday,False
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member,2019-12-31,23:59:37,December,Monday,weekday,False
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,2019-12-31,23:59:38,December,Monday,weekday,False


In [17]:
y2019.loc[y2019['Holiday'] == False, 'holiday'] = 'No'
y2019.loc[y2019['Holiday'] == True, 'holiday'] = 'Yes'

In [18]:
y2019[['hour','minutes','second']] = y2019['time'].astype(str).str.split(':', expand=True).astype(int)
y2019

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,date,time,month,weekday,workingday,Holiday,holiday,hour,minutes,second
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,2019-01-01,00:04:48,January,Monday,weekday,False,No,0,4,48
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,2019-01-01,00:06:37,January,Monday,weekday,False,No,0,6,37
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual,2019-01-01,00:08:46,January,Monday,weekday,False,No,0,8,46
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member,2019-01-01,00:08:47,January,Monday,weekday,False,No,0,8,47
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member,2019-01-01,00:12:29,January,Monday,weekday,False,No,0,12,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161873,130,2019-12-31 23:57:36,2019-12-31 23:59:46,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member,2019-12-31,23:57:36,December,Monday,weekday,False,No,23,57,36
161874,664,2019-12-31 23:57:47,2020-01-01 00:08:51,31125,15th & W St NW,31281,8th & O St NW,W24197,Member,2019-12-31,23:57:47,December,Monday,weekday,False,No,23,57,47
161875,389,2019-12-31 23:59:37,2020-01-01 00:06:06,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member,2019-12-31,23:59:37,December,Monday,weekday,False,No,23,59,37
161876,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,2019-12-31,23:59:38,December,Monday,weekday,False,No,23,59,38


In [19]:
g1 = y2019.groupby([ 'date', 'hour','weekday','holiday']).size()

In [20]:
g2 = g1.reset_index()
g2.columns = ['date', 'hour','weekday','holiday','count']
g2

,date,hour,weekday,holiday,count
0,2019-01-01,0,Monday,No,60
1,2019-01-01,1,Monday,No,107
2,2019-01-01,2,Monday,No,102
3,2019-01-01,3,Monday,No,25
4,2019-01-01,4,Monday,No,14
...,...,...,...,...,...
7998,2019-12-31,19,Monday,No,180
7999,2019-12-31,20,Monday,No,135
8000,2019-12-31,21,Monday,No,111
8001,2019-12-31,22,Monday,No,80


In [21]:
X = g2[['hour','weekday','holiday']]
X

,hour,weekday,holiday
0,0,Monday,No
1,1,Monday,No
2,2,Monday,No
3,3,Monday,No
4,4,Monday,No
...,...,...,...
7998,19,Monday,No
7999,20,Monday,No
8000,21,Monday,No
8001,22,Monday,No


In [22]:
Y = g2['count']
Y

0        60
1       107
2       102
3        25
4        14
       ... 
7998    180
7999    135
8000    111
8001     80
8002    107
Name: count, Length: 8003, dtype: int64

In [23]:
X.dtypes

hour        int64
weekday    object
holiday    object
dtype: object

In [24]:
X = pd.get_dummies(X)
X

,hour,weekday_0,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,holiday_No,holiday_Yes
0,0,0,0,1,0,0,0,0,1,0
1,1,0,0,1,0,0,0,0,1,0
2,2,0,0,1,0,0,0,0,1,0
3,3,0,0,1,0,0,0,0,1,0
4,4,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
7998,19,0,0,1,0,0,0,0,1,0
7999,20,0,0,1,0,0,0,0,1,0
8000,21,0,0,1,0,0,0,0,1,0
8001,22,0,0,1,0,0,0,0,1,0


In [25]:
regressor = LinearRegression()

In [26]:
regressor.fit(X,Y)

LinearRegression()

In [27]:
pickle.dump(regressor, open('model.pkl', 'wb'))

In [28]:
model = pickle.load(open('model.pkl', 'rb'))

In [29]:
data = []

In [30]:
item = [0, 'No', 'Monday']